In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# print(os.getcwd())
# os.chdir("/content/drive/MyDrive/phys310-1/310MLP/src")
# !pip install multiprocess

In [ ]:
from importlib import reload
import model
model = reload(model)

import tf_keras as keras
import pandas as pd
import tensorflow as tf
import crystal_loader
from tqdm import tqdm
import tqdm.keras
import numpy as np
from symmetry import *
import dill
from sklearn.preprocessing import StandardScaler
import h5py
from sklearn.model_selection import train_test_split


import tensorflow as tf

try:
  physical_devices = tf.config.list_physical_devices('GPU')

  print("GPU:", tf.config.list_physical_devices('GPU'))
  print("Num GPUs:", len(physical_devices))
  # tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  print("No GPU")

In [ ]:
import os
print(os.getcwd())

features_path = "../pickles/TiO2_2015_angfixed_x2_features.h5"
labels_path = "../pickles/TiO2_2015_angfixed_x2_labels.h5"

with h5py.File(features_path, "r") as f:
    features = [f[f"array_{i}"][:] for i in range(len(f))]

with h5py.File(labels_path, "r") as f:
    labels = [f[f"array_{i}"][:] for i in range(len(f))]

labels = np.array(labels)

In [ ]:
layers = [keras.layers.Dense(30, activation="ReLU"),
          keras.layers.Dense(30, activation="ReLU")]

MLP1 = model.MLPNet(layers=layers, N_features=70, ragged_processing=False)

MLP1.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0003),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.RootMeanSquaredError()]
)


In [ ]:
def scale_ragged(features):
    stacked = np.vstack(features)
    SSC = StandardScaler().fit(stacked)
    scaled_features = [SSC.transform(struct) for struct in features]

    return scaled_features

scaled_features = scale_ragged(features)


scaled_features = features

In [ ]:
Xtrain, Xtest, y_train, y_test = train_test_split(scaled_features, labels, shuffle=True, random_state=12, test_size=0.2)
Xval, Xtest, y_val, y_test = train_test_split(Xtest, y_test, shuffle=True, random_state=12, test_size=0.5)

Xtrain = tf.ragged.constant(Xtrain, ragged_rank=1, inner_shape=(70,))
Xval = tf.ragged.constant(Xval, ragged_rank=1, inner_shape=(70,))
Xtest = tf.ragged.constant(Xtest, ragged_rank=1, inner_shape=(70,))


In [ ]:
from sklearn.neighbors import KernelDensity

def inverse_density_weights(y, **kwargs):
    kde = KernelDensity(kernel='gaussian', bandwidth=50.0).fit(y.reshape(-1, 1))

    # 2. Estimate density
    log_density = kde.score_samples(y.reshape(-1, 1))
    density = np.exp(log_density)

    # 3. Invert density for weights
    weights = 1 / (density + 1e-6)
    weights /= np.mean(weights)  # optional normalization

    return weights.reshape(-1, 1)


In [ ]:
# y_train_weights = inverse_density_weights(y_train)
# y_val_weights = inverse_density_weights(y_val)

In [ ]:
import os

checkpoint_filepath = "./training_f4x_weighted/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_filepath)
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_root_mean_squared_error',
    mode='max',
    save_best_only=True)

# import matplotlib.pyplot as plt

# fig, ax = plt.subplots()

# ax.scatter(y_train, 0.001 * y_train_weights / np.max(y_train_weights))
# ax.hist(y_train, density=True, bins="auto", color="orange");

In [ ]:
res1 = MLP1.fit(
    Xtrain, y_train,# sample_weight=y_train_weights,
    batch_size = 48,
    epochs = 3000,
    validation_freq = 1,
    verbose = 0,
    validation_data=(Xval, y_val),
    callbacks=[model_checkpoint_callback],
)

In [ ]:
# res2 = MLP1.fit(
#     Xtrain, y_train,# sample_weight=y_train_weights,
#     batch_size = 20,
#     epochs = 1,
#     validation_freq = 1,
#     verbose = 0,
#     validation_data=(Xval, y_val),
#     callbacks=[model_checkpoint_callback],
# )

In [ ]:
res = res1
print(res.history.keys())

In [ ]:
import matplotlib.pyplot as plt

plt.close(1)
fig, ax = plt.subplots(1, 2, num=1, figsize=(10, 5))
ax[0].plot(res.epoch, res.history["loss"], label="training")
ax[0].plot(res.epoch, res.history["val_loss"], label="validation")

ax[0].set_ylabel("loss (MSE)")

ax[1].plot(res.epoch, res.history["root_mean_squared_error"], label="training")
ax[1].plot(res.epoch, res.history["val_root_mean_squared_error"], label="validation")
ax[1].set_yscale("log")
ax[0].set_yscale("log")
ax[1].set_ylabel("loss (RMSE), eV")



ax[0].set_xlabel("epoch")
ax[1].set_xlabel("epoch")

ax[0].legend()
ax[1].legend()

In [ ]:
ax01 = ax[0].inset_axes([0.5, 0.5, 0.5, 0.4])
ax01.plot(res.epoch, res.history["loss"])
ax01.plot(res.epoch, res.history["val_loss"], label="validation")
ax01.set_xlim(150, 200)
ax01.set_ylim(0,80000)

ax02 = ax[1].inset_axes([0.5, 0.5, 0.5, 0.4])
ax02.plot(res.epoch, res.history["root_mean_squared_error"])
ax02.plot(res.epoch, res.history["val_root_mean_squared_error"], label="validation")
ax02.set_xlim(150, 200)
ax02.set_ylim(0,500)

In [ ]:
y_pred = MLP1.predict(Xtest)
y_exam = np.squeeze(y_test)


filt_80k = (y_exam >= -9e4) & (y_exam <= -7e4)
filt_40k = (y_exam >= -5e4) & (y_exam <= -3e4)
filt_20k = (y_exam >= -3e4) & (y_exam <= -1e4)
filt_10k = (y_exam >= -1e4) & (y_exam <= 1e4)

yt_80k = y_exam[filt_80k]
yt_40k = y_exam[filt_40k]
yt_20k = y_exam[filt_20k]
yt_10k = y_exam[filt_10k]

y_pred_80k = y_pred[filt_80k]
y_pred_40k = y_pred[filt_40k]
y_pred_20k = y_pred[filt_20k]
y_pred_10k = y_pred[filt_10k]

In [ ]:
fig, ax = plt.subplots(1, 4, num=2, figsize=(20, 5))

for axes, ytest, ypred in zip(ax, [yt_80k, yt_40k, yt_20k, yt_10k], [y_pred_80k, y_pred_40k, y_pred_20k, y_pred_10k]):
    print(len(ytest), len(ypred))
    axes.hist(ytest, bins="auto", label="true", alpha=0.6)
    axes.hist(ypred, bins="auto", label="predicted", alpha=0.7)

    axes.legend()

In [ ]:
from sklearn.metrics import r2_score, root_mean_squared_error

fig, ax = plt.subplots(num=3, figsize=(10, 6))
ax.scatter(y_exam, y_pred)
ax.set_title("y pred vs y test overview")

fig, ax = plt.subplots(1, 4, num=2, figsize=(20, 5))
fig.suptitle("Predicted vs Actual plots in various energy ranges")
print(len(y_exam))
for axes, ytest, ypred in zip(ax, [yt_80k, yt_40k, yt_20k, yt_10k], [y_pred_80k, y_pred_40k, y_pred_20k, y_pred_10k]):
    axes.scatter(ytest, ypred, lw=0, marker=".", c=np.arange(np.size(ytest)))
    axes.set_xlabel(f"true (eV) (N={len(ytest)})")
    axes.set_ylabel("predicted (eV)")
    axes.set_aspect("equal")
    axmin = np.min([np.min(ytest), np.min(ypred)])
    axmax = np.max([np.max(ytest), np.max(ypred)])
    x = np.linspace(axmin, axmax)
    axes.plot(x, x, c="r")
    axes.set_xlim(axmin, axmax)
    axes.set_ylim(axmin, axmax)

print(r2_score(y_exam, y_pred))
print(root_mean_squared_error(y_exam, y_pred))

In [ ]:
### prompt: save MLP1's weights and training history
import os
print(os.getcwd())
# Save MLP1's weights
MLP1.save_weights('./saved_models/f4x_weighted.h5')

# Save training history
with open('./saved_models/f4x_weighted.dill', 'wb') as file_pi:
    dill.dump(res.history, file_pi)


In [ ]:

MLP1 = model.MLPNet(layers=layers, N_features=70, ragged_processing=False)
MLP1.built = True
MLP1.load_weights('./saved_models/f3x_weighted.h5')

In [ ]:
import pandas as pd
import crystal_loader

structures = []
LVs = []

LV = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
# x = pd.DataFrame({"Atom": ["Ti", "O", "Ti"], "x": [0, d, 0.01], "y": [0, 0, 0.9], "z": [0, 0, 0]})
for d in np.linspace(0.001, 1, 1200):
    x = pd.DataFrame({"Atom": ["Ti", "O", "Ti"], "x": [0, d, 1], "y": [0, 0, 0], "z": [0, 0, 0]})
    # print(x)
    structures.append(x)
    LVs.append(LV)

LVs = np.array(LVs)

In [ ]:
Rc = 6.5 # A
Rs = 0

# build radial basis params:

radial_etas = [0.003214, 0.035711, 0.071421, 0.124987, 0.214264, 0.357106, 0.714213, 1.428426]
params_radial = tuple([{"Rs": Rs, "eta": eta} for eta in radial_etas])

angular_etas = [0.000357, 0.028569, 0.089277, 0.000357, 0.028569, 0.089277, 0.000357, 0.028569, 0.089277] * 2
angular_lambdas = [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
angular_zetas = [1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0]

params_angular = tuple([{"Rs": Rs, "eta": eta, "lmbda": lam, "zeta": zeta} for eta, lam, zeta in zip(angular_etas, angular_lambdas, angular_zetas)])

tio_params_ang = {("Ti", "Ti"): params_angular, ("Ti", "O"): params_angular, ("O", "O"): params_angular}
tio_params_rad = {"Ti": params_radial, "O": params_radial}

In [ ]:
# features = crystal_loader.build_features(structures, LVs, Rc=6.5, params_rad=tio_params_rad, params_ang=tio_params_ang)
import symmetry

# features = symmetry.get_features(structures, LVs, Rc=6.5, params_rad=tio_params_rad, params_ang=tio_params_ang)

features = crystal_loader.build_features(structures, LVs, Rc=6.5, params_rad=tio_params_rad, params_ang=tio_params_ang)

In [ ]:
feats = tf.ragged.constant(features, ragged_rank=1, inner_shape=(70,)) / 2
y_pred = MLP1.predict(feats)


In [ ]:
plt.close(15)
d = np.linspace(0.001, 1, 1200)
fig, ax = plt.subplots(num=15)
ax.plot(d, y_pred)
ax.set_xlabel("middle atom position (angstroms)")
ax.set_ylabel('predicted E (eV)')


In [ ]:
eggs = ("bee", "gee", "liz", "cow")

A, B, C, *rest = eggs
D = rest[0] if rest else None